In [1]:
import pyscf
from pyscf import tools, symm
import numpy as np


In [3]:
scf_energy=[]
molecule = """
    N   0.0 0.0 0.0
    N   2 0.0 0.0
    """
print(molecule)
basis = "sto3g"
mol = pyscf.gto.Mole(
    atom    =   molecule,
    symmetry=   True,
    spin    =   0,
    charge  =   0,
    basis   =   basis)


mol.build()
print("symmetry: ",mol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
mf = pyscf.scf.ROHF(mol)
mf.verbose = 4
mf.conv_tol = 1e-8
mf.conv_tol_grad = 1e-5
mf.chkfile = "scf.fchk"
mf.init_guess = "sad"
mf.run(max_cycle=200)

print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
scf_energy.append(mf.e_tot)
print(scf_energy)
norb = mf.mo_coeff.shape[0]
    #print(mf.mo_coeff)
print(norb)
pyscf.tools.molden.from_mo(mol, "C_RHF_n2.molden", mf.mo_coeff)
for s,i,c in zip(mol.irrep_name, mol.irrep_id, mol.symm_orb):
    print(" %5s %3i %4i"%(s, i, c.shape[1]))

def myocc(mf):    
    mol = mf.mol
    irrep_id = mol.irrep_id
    so = mol.symm_orb
    orbsym = symm.label_orb_symm(mol, irrep_id, so, mf.mo_coeff)
    occsym = np.array(orbsym)[mf.mo_occ==2]
    virsym = np.array(orbsym)[mf.mo_occ==0]
    for ir,irname in enumerate(mol.irrep_name):
        print('%5s, Occ = %5d, Vir = %d' % (irname, sum(occsym==ir), sum(virsym==ir)))
        print(orbsym)
        print(mol.irrep_id)
        print(mol.irrep_name)
myocc(mf)
    

        




    N   0.0 0.0 0.0
    N   2 0.0 0.0
    
symmetry:  Dooh


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 130 MB)
num. doubly occ = 7  num. singly occ = 0
init E= -106.24556838672
HOMO (E1ux) = -0.13495270980812  LUMO (E1gy) = -0.0553429789815831
cycle= 1 E= -106.870762526945  delta_E= -0.625  |g|= 0.0361  |ddm|=  1.6
HOMO (E1ux) = -0.235961538132484  LUMO (E1gy) = 0.0775472451690116
cycle= 2 E= -106.871458336518  delta_E= -0.000696  |g|= 0.0088  |ddm|= 0.0404
HOMO (E1ux) = -0.238176852623986  LUMO (E1gy) = 0.075729199638266
cycle= 3 E= -106.871504043906  delta_E= -4.57e-05  |g|= 5.55e-05  |ddm|= 0.0133
HOMO (E1ux) = -0.2381789

In [8]:
sym_map = {}
for ir,irname in enumerate(mol.irrep_id):
    sym_map[irname] = ir
    
clusters = []
for i in range(8):
    clusters.append([])
frozen= [0,1]  
orb_syms = {}
for i in range(norb):
        # print(" Orb: %5i %5s %5s " %(i, mf.orbsym[i], mf.orbsym[i]))
    print(" Orb: %5i %5s " %(i,  mol.irrep_name[sym_map[mf.orbsym[i]]]))
        # orb_syms.append(mol.irrep_name[sym_map[mf.orbsym[i]]])
    name = mol.irrep_name[sym_map[mf.orbsym[i]]]
    if i not in frozen:
        if name == "A1g" or name == "A1u":
            clusters[1].append(i)
        elif name == "E1gx" or name == "E1ux":
            clusters[2].append(i)
        elif name == "E1gy" or name == "E1uy":
            clusters[3].append(i)
if name in orb_syms:
    orb_syms[name].append(i)
else:
    orb_syms[name] = [i]    
            
    #specify frozen core
clusters[0] = [0,1]
C=mf.mo_coeff
C_ordered = np.zeros((norb,0))
for ci,c in enumerate(clusters):
    C_ordered = np.hstack((C_ordered, C[:,c]))

 Orb:     0   A1u 
 Orb:     1   A1g 
 Orb:     2   A1g 
 Orb:     3   A1u 
 Orb:     4   A1g 
 Orb:     5  E1uy 
 Orb:     6  E1ux 
 Orb:     7  E1gx 
 Orb:     8  E1gy 
 Orb:     9   A1u 


In [6]:
pyscf.tools.molden.from_mo(mol, "C_ordered_n2.molden", C_ordered)
init_fspace = [(2,2), (3,3), (1,1), (1,1)]
print(clusters)
print(init_fspace)
for ci in clusters:
    print(len(ci))
pyscf.scf.hf_symm.analyze(mf)
h0 = pyscf.gto.mole.energy_nuc(mol)
h1 = pyscf.scf.hf.get_hcore(mol)
h1 = C_ordered.T @ h1 @ C_ordered

nact = h1.shape[0]
h2 = pyscf.ao2mo.kernel(mol, C_ordered, aosym="s4", compact=False)
h2.shape = (nact, nact, nact, nact)

S = mol.intor("int1e_ovlp_sph")

[[0, 1], [2, 3, 4, 9], [6, 7], [5, 8], [], [], [], []]
[(1, 1), (4, 4), (1, 1), (1, 1)]
2
4
2
2
0
0
0
0
**** SCF Summaries ****
Total Energy =                        -106.871504045608347
Nuclear Repulsion Energy =              12.964841667540002
One-electron Energy =                 -172.985397613440711
Two-electron Energy =                   53.149051900292356
Wave-function symmetry = Dooh
occupancy for each irrep:    A1g E1gx E1gy  A1u E1uy E1ux
                               3    0    0    2    1    1
**** MO energy ****
MO #1 (A1u #1), energy= -15.3760078772939 occ= 2
MO #2 (A1g #1), energy= -15.3757529572774 occ= 2
MO #3 (A1g #2), energy= -0.970273717659548 occ= 2
MO #4 (A1u #2), energy= -0.82789632326717 occ= 2
MO #5 (A1g #3), energy= -0.31735752957615 occ= 2
MO #6 (E1uy #1), energy= -0.238179366868072 occ= 2
MO #7 (E1ux #1), energy= -0.238179366868072 occ= 2
MO #8 (E1gx #1), energy= 0.0757293582220333 occ= 0
MO #9 (E1gy #1), energy= 0.0757293582220338 occ= 0
MO #10 (A1u #3), ene

In [9]:
pyscf.scf.hf_symm.analyze(mf)

**** SCF Summaries ****
Total Energy =                        -106.871504045608347
Nuclear Repulsion Energy =              12.964841667540002
One-electron Energy =                 -172.985397613440711
Two-electron Energy =                   53.149051900292356
Wave-function symmetry = Dooh
occupancy for each irrep:    A1g E1gx E1gy  A1u E1uy E1ux
                               3    0    0    2    1    1
**** MO energy ****
MO #1 (A1u #1), energy= -15.3760078772939 occ= 2
MO #2 (A1g #1), energy= -15.3757529572774 occ= 2
MO #3 (A1g #2), energy= -0.970273717659548 occ= 2
MO #4 (A1u #2), energy= -0.82789632326717 occ= 2
MO #5 (A1g #3), energy= -0.31735752957615 occ= 2
MO #6 (E1uy #1), energy= -0.238179366868072 occ= 2
MO #7 (E1ux #1), energy= -0.238179366868072 occ= 2
MO #8 (E1gx #1), energy= 0.0757293582220333 occ= 0
MO #9 (E1gy #1), energy= 0.0757293582220338 occ= 0
MO #10 (A1u #3), energy= 0.220543346732608 occ= 0
 ** MO coefficients (expansion on meta-Lowdin AOs) **
               #1(A1

((array([1.9999958 , 1.97146909, 1.02853511, 1.        , 1.        ,
         1.9999958 , 1.97146909, 1.02853511, 1.        , 1.        ]),
  array([-1.24344979e-14,  3.55271368e-15])),
 array([-1.80601945e-14,  1.25000218e-48, -1.81123345e-32]))

In [10]:
np.save("integrals_h0_n2", h0)
np.save("integrals_h1_n2", h1)
np.save("integrals_h2_n2", h2)
np.save("mo_coeffs_n2",C_ordered)
np.save("overlap_mat_n2", S)
np.save("clusters_n2", clusters)
np.save("init_fspace_n2", init_fspace)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.

In [11]:
scf_energy=[]
for ri in range(0,40):
    print(ri)
    ###     PYSCF INPUT
    r0 = 0.7 + 0.08 * ri
    molecule = """
    N   0.0 0.0 0.0
    N   {} 0.0 0.0
    """.format(r0,r0/2)
    print(molecule)
    basis = "sto3g"
    mol = pyscf.gto.Mole(
    atom    =   molecule,
    symmetry=   True,
    spin    =   0,
    charge  =   0,
    basis   =   basis)
    mol.build()
    print("symmetry: ",mol.topgroup)
# mf = pyscf.scf.UHF(pymol).x2c()
    mf = pyscf.scf.ROHF(mol)
    mf.verbose = 4
    mf.conv_tol = 1e-8
    mf.conv_tol_grad = 1e-5
    mf.chkfile = "scf.fchk"
    mf.init_guess = "sad"
    mf.run(max_cycle=200)

    print(" Hartree-Fock Energy: %12.8f" % mf.e_tot)
    scf_energy.append(mf.e_tot)
    print(scf_energy)
    norb = mf.mo_coeff.shape[0]
    #print(mf.mo_coeff)
    print(norb)
    pyscf.tools.molden.from_mo(mol, "C_RHF_n2.molden", mf.mo_coeff)
    for s,i,c in zip(mol.irrep_name, mol.irrep_id, mol.symm_orb):
        print(" %5s %3i %4i"%(s, i, c.shape[1]))
    def myocc(mf):
        mol=mf.mol
        irrep_id = mol.irrep_id
        so = mol.symm_orb
        orbsym = symm.label_orb_symm(mol, irrep_id, so, mf.mo_coeff)
        occsym = np.array(orbsym)[mf.mo_occ==2]
        virsym = np.array(orbsym)[mf.mo_occ==0]
        for ir,irname in enumerate(mol.irrep_name):
            print('%5s, Occ = %5d, Vir = %d' % (irname, sum(occsym==ir), sum(virsym==ir)))
            print(orbsym)
            print(mol.irrep_id)
            print(mol.irrep_name)
    myocc(mf)
    
    sym_map = {}
    for ir,irname in enumerate(mol.irrep_id):
        sym_map[irname] = ir
    
    clusters = []
    for i in range(8):
        clusters.append([])
    frozen= [0,1]  
    orb_syms = {}
    for i in range(norb):
        # print(" Orb: %5i %5s %5s " %(i, mf.orbsym[i], mf.orbsym[i]))
        print(" Orb: %5i %5s " %(i,  mol.irrep_name[sym_map[mf.orbsym[i]]]))
        # orb_syms.append(mol.irrep_name[sym_map[mf.orbsym[i]]])
        name = mol.irrep_name[sym_map[mf.orbsym[i]]]
        if i not in frozen:
            if name == "A1g" or name == "A1u":
                clusters[1].append(i)
            elif name == "E1gx" or name == "E1ux":
                clusters[2].append(i)
            elif name == "E1gy" or name == "E1uy":
                clusters[3].append(i)
    if name in orb_syms:
        orb_syms[name].append(i)
    else:
        orb_syms[name] = [i]
    #specify frozen core
    clusters[0] = [0,1]
    C=mf.mo_coeff
    C_ordered = np.zeros((norb,0))
    for ci,c in enumerate(clusters):
        C_ordered = np.hstack((C_ordered, C[:,c]))
        

    pyscf.tools.molden.from_mo(mol, "C_ordered_n2.molden", C_ordered)
    init_fspace = [(1,1), (4,4), (1,1), (1,1)]
    print(clusters)
    print(init_fspace)
    for ci in clusters:
        print(len(ci))
    pyscf.scf.hf_symm.analyze(mf)
    h0 = pyscf.gto.mole.energy_nuc(mol)
    h1 = pyscf.scf.hf.get_hcore(mol)
    h1 = C_ordered.T @ h1 @ C_ordered

    nact = h1.shape[0]

    h2 = pyscf.ao2mo.kernel(mol, C_ordered, aosym="s4", compact=False)
    h2.shape = (nact, nact, nact, nact)

    S = mol.intor("int1e_ovlp_sph")
    

    

0

    N   0.0 0.0 0.0
    N   0.7 0.0 0.0
    
symmetry:  Dooh


******** <class 'pyscf.scf.hf_symm.SymAdaptedROHF'> ********
method = SymAdaptedROHF-ROHF-RHF
initial guess = sad
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-08
SCF conv_tol_grad = 1e-05
SCF max_cycles = 200
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = scf.fchk
max_memory 4000 MB (current use 137 MB)
num. doubly occ = 7  num. singly occ = 0
init E= -108.97174017784
HOMO (A1g) = -0.393295716674018  LUMO (E1gy) = 0.425204690326911
cycle= 1 E= -105.697174311695  delta_E= 3.27  |g|= 0.0397  |ddm|= 2.06
HOMO (A1g) = -0.652901487676409  LUMO (E1gy) = 0.53458652363543
cycle= 2 E= -105.697408128486  delta_E= -0.000234  |g|= 0.00518  |ddm|= 0.101
HOMO (A1g) = -0.652802043525838  LUMO (E1gx) = 0.535326838501894
cycle= 3 E= -105.697412212734  delta_E= -4.08e-06  |g|= 5.39e-06  |ddm|= 0.0149
HOMO (A1g) = -0.652800730327